In [2]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 
from spark_util import get_parser


from get_missing_articles import *
import os
import codecs
import networkx as nx
from collections import Counter
from pprint import pprint
from ConfigParser import SafeConfigParser
import pandas as pd

In [3]:
s = 'simple'
t = 'es'

cp = SafeConfigParser()
cp.read('/home/ellery/translation-recs-app/translation-recs.ini')

delim = '|'

outerdir = os.path.join(cp.get('general', 'local_data_dir'), 'translation-recs-app/data', s)
if not os.path.exists(outerdir):
    os.makedirs(outerdir)

innerdir = os.path.join(cp.get('general', 'local_data_dir'), 'translation-recs-app/data', s, t)
if not os.path.exists(innerdir):
    os.makedirs(innerdir)


wd_languages = set([s, t])
rd_languages = set([s, t, 'wikidata'])
ill_languages_from = set([s, t])
ill_languages_to = set([s, t])



G = create_graph(sc, cp, delim, wd_languages, rd_languages, ill_languages_from, ill_languages_to)
print "Got entire Graph"

get_missing_items(sc, cp, G, s, t, delim, os.path.join(innerdir, cp.get('missing', 'missing_items')))
print "Got missing Items"